In [1]:
import random
import numpy as np
import gurobipy as gp

# Room draw optimization models using gurobi python API
## 1. Model for multiple preferences with individual parties
Consistent with the output in multi_preferences model in `multi_preferences.mod`

In [2]:
# number parameters
n_rooms = 20
n_people = 20
n_preferences = 5

ROOMS = list(range(1, n_rooms + 1))
PEOPLE = list(range(1, n_people + 1))
PREFERENCES = list(range(1, n_preferences + 1))

# preference and weight parameters
weights = {(p, pr): n_preferences - pr  + 1 for p in PEOPLE for pr in PREFERENCES}



# preferences = np.random.randint(low=1,high=n_rooms,size=(n_people, n_preferences))
preferences = np.array([
       [ 5,  7, 19, 13,  9],
       [16,  6,  3,  6, 18],
       [14,  2, 15,  7, 14],
       [15,  2,  4,  2, 16],
       [18,  5, 18, 13,  5],
       [ 3,  1,  5, 15,  3],
       [ 2,  5,  8,  6,  9],
       [13,  9, 12,  1,  5],
       [19, 17,  6, 13, 10],
       [ 5, 17, 19,  5, 14],
       [ 5, 17, 12,  1, 15],
       [16, 18,  4,  4,  4],
       [ 9,  8, 13,  4, 15],
       [ 6,  8,  3, 11, 14],
       [12, 18, 19, 16, 14],
       [17,  4, 12,  6, 13],
       [16, 17, 13,  3, 14],
       [ 3, 12, 17,  4,  1],
       [ 4, 12,  3, 11, 11],
       [14, 14, 16, 12, 18]])


[[ 7  8 20 17  9]
 [18  6  5  6 20]
 [14  3 17  8 14]
 [17  4  7  3 20]
 [19  8 20 13  9]
 [ 6  4  9 18  5]
 [ 6  7 10  8 13]
 [13 13 15  2  6]
 [20 19  9 16 10]
 [ 9 18 20  9 18]
 [ 7 18 15  5 16]
 [19 18  6  4  6]
 [11 11 15  6 16]
 [ 9 10  3 13 18]
 [16 20 19 18 15]
 [19  5 14  6 16]
 [17 18 17  4 16]
 [ 7 12 20  4  4]
 [ 4 13  3 15 12]
 [15 16 18 16 19]]


dtype('int64')

In [3]:
m = gp.Model("Multi-Preference Model")
satisfied = m.addVars(PEOPLE, PREFERENCES, vtype=gp.GRB.BINARY)
assignment = m.addVars(PEOPLE, ROOMS, vtype=gp.GRB.BINARY)
room_number = m.addVars(PEOPLE, lb=1, vtype=gp.GRB.INTEGER)

m.addConstrs((satisfied[p,pr] == assignment[p,preferences[p-1,pr-1]] for p in PEOPLE for pr in PREFERENCES),name='Preference Satisfied')
m.addConstrs((assignment.sum('*', r) == 1 for r in ROOMS), name='No overlaps')

rooms_dict = {(p, r): r for p in PEOPLE for r in ROOMS }
m.addConstrs((assignment.prod(rooms_dict, p, '*') == room_number[p] for p in PEOPLE)
, name='Assignment number')

m.setObjective(satisfied.prod(weights, '*', '*'), gp.GRB.MAXIMIZE)


Restricted license - for non-production use only - expires 2024-10-28


In [4]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 140 rows, 520 columns and 1020 nonzeros
Model fingerprint: 0x4c7a5c8a
Variable types: 0 continuous, 520 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 100 rows and 120 columns
Presolve time: 0.01s
Presolved: 40 rows, 400 columns, 800 nonzeros
Variable types: 0 continuous, 400 integer (400 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 9.600000e+01, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0        

## 2. Model for range preferences
In this model, we try to implement preferences for not a particular room but a general 

In [20]:
preferences_lower = preferences
#preferences_upper = np.minimum(preferences + np.random.randint(0, 5, (20, 5)), 20*np.ones((20, 5))).astype(np.int64)
preferences_upper = np.array(
    [
        [ 5, 11, 20, 17,  9],
        [16,  9,  7,  9, 20],
        [16,  3, 16,  8, 15],
        [16,  4,  5,  5, 17],
        [18,  5, 18, 16,  8],
        [ 3,  5,  5, 16,  6],
        [ 2,  8, 10,  6, 13],
        [16, 12, 14,  2,  8],
        [19, 19,  9, 14, 14],
        [ 8, 17, 19,  6, 17],
        [ 9, 20, 13,  2, 16],
        [18, 20,  7,  4,  4],
        [11,  9, 13,  5, 18],
        [ 9, 12,  5, 15, 14],
        [15, 19, 20, 17, 16],
        [17,  6, 16,  8, 17],
        [17, 20, 15,  5, 16],
        [ 3, 12, 18,  6,  5],
        [ 8, 14,  6, 11, 14],
        [17, 14, 17, 16, 20]])

In [52]:
preferences_upper = np.array([[ 5, 11, 20, 17,  9],
       [16,  9,  7,  9, 20],
       [16,  3, 16,  8, 15],
       [16,  4,  5,  5, 17],
       [18,  5, 18, 16,  8],
       [ 3,  5,  5, 16,  6],
       [ 2,  8, 10,  6, 13],
       [16, 12, 14,  2,  8],
       [19, 19,  9, 14, 14],
       [ 8, 17, 19,  6, 17],
       [ 9, 20, 13,  2, 16],
       [18, 20,  7,  4,  4],
       [11,  9, 13,  5, 18],
       [ 9, 12,  5, 15, 14],
       [15, 19, 20, 17, 16],
       [17,  6, 16,  8, 17],
       [17, 20, 15,  5, 16],
       [ 3, 12, 18,  6,  5],
       [ 8, 14,  6, 11, 14],
       [17, 14, 17, 16, 20]])

In [54]:
m = gp.Model("Multi-Preference Model")
satisfied = m.addVars(PEOPLE, PREFERENCES, vtype=gp.GRB.BINARY)
assignment = m.addVars(PEOPLE, ROOMS, vtype=gp.GRB.BINARY)
room_number = m.addVars(PEOPLE, lb=1, vtype=gp.GRB.INTEGER)

p = m.addConstrs((satisfied[p,pr] == gp.quicksum(assignment[p,r] for r in range(preferences_lower[p-1,pr-1], preferences_upper[p-1,pr-1] + 1)) for p in PEOPLE for pr in PREFERENCES),name='Preference Satisfied')
m.addConstrs((assignment.sum('*', r) == 1 for r in ROOMS), name='No overlaps')

rooms_dict = {(p, r): r for p in PEOPLE for r in ROOMS }
m.addConstrs((assignment.prod(rooms_dict, p, '*') == room_number[p] for p in PEOPLE)
, name='Assignment number')

m.setObjective(satisfied.prod(weights, '*', '*'), gp.GRB.MAXIMIZE)

In [55]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 140 rows, 520 columns and 1203 nonzeros
Model fingerprint: 0xcb7b9642
Variable types: 0 continuous, 520 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 31 rows and 120 columns
Presolve time: 0.00s
Presolved: 109 rows, 400 columns, 1039 nonzeros
Variable types: 0 continuous, 400 integer (400 binary)

Root relaxation: objective 1.410000e+02, 119 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     141.0000000  141.00000  0.00%   

## 3. Model for macro groups
In this model, consider preferences for dorm suites

## 4. Model for realistic draw order
In thin model, consider the ordering and preference of selection